In [ ]:
from datetime import datetime
import os
import pandas as pd
import numpy as np
import holidays
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model
from sklearn.metrics import mean_absolute_error, r2_score

## Decision Making:
- Since openweather only offer hourly prediction weather data, we predict bikes data hourly
- To make the model be able to capture the hour feature's nature, we use cyclical encoding for hour feature
- Since we are going to train each model for each station, we drop station_id and capacity in each station's data
- We estimate average temperature, relative humidity and barometric pressure from max, min and standard deviation and merge them averagely into each hour

## Chosen features:
- hour
- temperature_celsius
- relative_humidity_percent
- barometric_pressure_hpa
- is_holiday
- is_weekend

## Predictions:
- num_bikes_available
- num_docks_available

In [ ]:
# Load the dataset
if os.path.isfile('training_data/final_merged_data.csv'):
    data = pd.read_csv("training_data/final_merged_data.csv")
    df = pd.DataFrame()
    
    # choose relevant features, time as index, station_id as index for each station data file
    df['time'] = data['last_reported']
    df['station_id'] = data['station_id']
    # estimate average temperature, relative humidity and barometric pressure from max, min and standard deviation
    df['temperature_celsius'] = (data['max_air_temperature_celsius'] + data['min_air_temperature_celsius']) / 2 + (data['air_temperature_std_deviation'] / np.sqrt(3))
    df['relative_humidity_percent'] = (data['max_relative_humidity_percent'] + data['min_relative_humidity_percent']) / 2 + (data['relative_humidity_std_deviation'] / np.sqrt(3))
    df['barometric_pressure_hpa'] = (data['max_barometric_pressure_hpa'] + data['min_barometric_pressure_hpa']) / 2 + (data['barometric_pressure_std_deviation'] / np.sqrt(3))
    # add is_holiday and is_weekend
    ireland_holidays = holidays.country_holidays('IE')
    df['is_holiday'] = data['last_reported'].apply(lambda x: x in ireland_holidays)
    df['is_weekend'] = data['last_reported'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S').weekday() in [5, 6])
    # target features
    df['available_bikes'] = data['num_bikes_available']
    df['available_docks'] = data['num_docks_available']
    
    # deal with each station's data by merging hourly data and imputing missing values
    for station_id in df['station_id'].unique():
        df_station = pd.DataFrame(df[df['station_id'] == station_id])
        # Resample data to hourly frequency and take the mean
        df_station['time'] = pd.to_datetime(df_station['time'])
        df_station.set_index('time', inplace=True)
        df_station = df_station.resample('h').mean()
        # Interpolate missing hourly values using the nearest available value
        df_station = df_station.interpolate(method='nearest')
        # Round the values after taking the mean for each hour
        df_station['available_bikes'] = df_station['available_bikes'].round()
        df_station['available_docks'] = df_station['available_docks'].round()
        df_station['temperature_celsius'] = df_station['temperature_celsius'].round(2)
        df_station['relative_humidity_percent'] = df_station['relative_humidity_percent'].round(2)
        df_station['barometric_pressure_hpa'] = df_station['barometric_pressure_hpa'].round(2)
        # station_id is constant column for each station data file, so drop it
        df_station.drop(columns=['station_id'], inplace=True)
        # Cyclical Encoding for hour feature
        df_station['hour_sin'] = np.sin(2 * np.pi * df_station.index.hour / 23.0).round(6)
        df_station['hour_cos'] = np.cos(2 * np.pi * df_station.index.hour / 23.0).round(6)
        # save the data for each station
        df_station.to_csv(f'training_data/station_{station_id}.csv', index=True)
        
    # station_70's data is corrupt, contain only 59 rows of data and all available_bikes and available_docks are 0
    # so I will not train the model on this station
    station_70 = data[data['station_id']==70]
    print(f"Number of rows for station 70: {len(station_70)}")
    if (station_70['num_bikes_available'] == 0).all() and (station_70['num_docks_available'] == 0).all():
        print("All available_bikes and available_docks are 0 for station 70")

In [ ]:
# example data for station 1
data = pd.read_csv(f'training_data/station_1.csv', parse_dates=['time'], index_col='time')
data

In [ ]:
# for station_id in range(1,118):
#     if (station_id == 70):
#         continue
#     df_station = pd.read_csv(f'station_{station_id}.csv')


In [ ]:
def visualize_prediction(y_test, predictions, mae_bikes, r2_score_bikes, mae_docks, r2_score_docks):
    # Display one prediction result and corresponding actual value
    print("Predictions:")
    print(predictions[-1])
    print("\nActual Values:")
    print(y_test[-1])
    
    # Plot results for available_bikes
    plt.figure(figsize=(10, 6))
    plt.plot(y_test[:, 0], label='Actual Bikes', color='green')
    plt.plot(predictions[:, 0], label='Predicted Bikes', color='red')
    plt.title(f'Available Bikes Forecast (MAE: {mae_bikes:.4f}, r2_score: {r2_score_bikes:.4f})')
    plt.xlabel('Hour Sequence')
    plt.ylabel('Available Bikes')
    plt.legend()
    plt.grid(True)
    plt.show()

    # Plot results for available_docks
    plt.figure(figsize=(10, 6))
    plt.plot(y_test[:, 1], label='Actual Docks', color='green')
    plt.plot(predictions[:, 1], label='Predicted Docks', color='red')
    plt.title(f'Available Docks Forecast (MAE: {mae_docks:.4f}, r2_score: {r2_score_docks:.4f})')
    plt.xlabel('Hour Sequence')
    plt.ylabel('Available Docks')
    plt.legend()
    plt.grid(True)
    plt.show()

In [12]:
# Load dataset
data = pd.read_csv(f'training_data/station_1.csv', parse_dates=['time'], index_col='time')

# Features and target variables
features = ['hour_sin', 'hour_cos','temperature_celsius', 'relative_humidity_percent', 'barometric_pressure_hpa', 'is_holiday', 'is_weekend']
targets = ['available_bikes', 'available_docks']

# Normalize the features
scaler = MinMaxScaler()
data[features] = scaler.fit_transform(data[features])

# Normalize the targets
target_scaler = MinMaxScaler()
data[targets] = target_scaler.fit_transform(data[targets])

# Move target columns to the last
data = data[[col for col in data.columns if col not in targets] + targets]

def split_train_test_validation(data):
    data = data.values
    train_data = data[:-24*6] # last 6 days for validation and testing
    validation_data = data[-24*6:-24*3] # last 3 days for validation
    test_data = data[-24*3:] # last 3 days for testing
    return train_data, validation_data, test_data

train_data, validation_data, test_data = split_train_test_validation(data)

In [18]:
data

,temperature_celsius,relative_humidity_percent,barometric_pressure_hpa,is_holiday,is_weekend,hour_sin,hour_cos,available_bikes,available_docks
time,,,,,,,,,
2024-12-01 00:00:00,0.961733,0.654483,0.449139,0.0,1.0,0.500000,1.000000,0.645161,0.322581
2024-12-01 01:00:00,0.961733,0.654483,0.449139,0.0,1.0,0.635214,0.981372,0.645161,0.322581
2024-12-01 02:00:00,0.961733,0.654483,0.449139,0.0,1.0,0.760399,0.926869,0.645161,0.322581
2024-12-01 03:00:00,0.867755,0.738454,0.412359,0.0,1.0,0.866272,0.840534,0.645161,0.354839
2024-12-01 04:00:00,0.867755,0.738454,0.412359,0.0,1.0,0.944980,0.728769,0.645161,0.354839
...,...,...,...,...,...,...,...,...,...
2024-12-31 19:00:00,0.707935,0.722648,0.387674,0.0,0.0,0.055020,0.728769,0.129032,0.870968
2024-12-31 20:00:00,0.675858,0.751050,0.383035,0.0,0.0,0.133728,0.840534,0.129032,0.870968
2024-12-31 21:00:00,0.655599,0.768091,0.379390,0.0,0.0,0.239601,0.926869,0.096774,0.903226


In [14]:
train_data.shape

(600, 9)

In [15]:
validation_data.shape

(72, 9)

In [17]:
test_data

array([[0.62577378, 0.84909854, 0.78628231, 0.        , 1.        ,
        0.5       , 1.        , 0.03225806, 0.96774194],
       [0.60326393, 0.77352433, 0.7851226 , 0.        , 1.        ,
        0.63521368, 0.98137175, 0.        , 1.        ],
       [0.62633652, 0.6621388 , 0.78793903, 0.        , 1.        ,
        0.76039899, 0.92686893, 0.        , 1.        ],
       [0.62633652, 0.65176587, 0.78595096, 0.        , 1.        ,
        0.86627178, 0.84053387, 0.        , 1.        ],
       [0.62633652, 0.65176587, 0.78595096, 0.        , 1.        ,
        0.94497975, 0.72876938, 0.        , 1.        ],
       [0.64490715, 0.63571252, 0.77683897, 0.        , 1.        ,
        0.99068579, 0.59986457, 0.        , 1.        ],
       [0.61564434, 0.76043468, 0.77087475, 0.        , 1.        ,
        1.        , 0.46337996, 0.12903226, 0.87096774],
       [0.61564434, 0.76043468, 0.77087475, 0.        , 1.        ,
        0.97223127, 0.32943719, 0.12903226, 0.87096774],


In [ ]:
visualize_training = True

# Load dataset
data = pd.read_csv(f'training_data/station_1.csv', parse_dates=['time'], index_col='time')

# Features and target variables
features = ['hour_sin', 'hour_cos','temperature_celsius', 'relative_humidity_percent', 'barometric_pressure_hpa', 'is_holiday', 'is_weekend']
targets = ['available_bikes', 'available_docks']

# Normalize the features
scaler = MinMaxScaler()
data[features] = scaler.fit_transform(data[features])

# Normalize the targets
target_scaler = MinMaxScaler()
data[targets] = target_scaler.fit_transform(data[targets])

# Move target columns to the last
data = data[[col for col in data.columns if col not in targets] + targets]

def split_train_test_validation(data):
    data = data.values
    train_data = data[:-24*6] # last 6 days for validation and testing
    validation_data = data[-24*6:-24*3] # last 3 days for validation
    test_data = data[-24*3:] # last 3 days for testing
    return train_data, validation_data, test_data
    

# Create sequences of data for LSTM
def create_sequences(data, seq_length):
    xs, ys = [], []
    for i in range(len(data) - seq_length):
        x = data[i:(i + seq_length), 0:-2]  # features
        y = data[i:(i + seq_length), -2:]   # targets
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

# Sequence length, 24 hours, from 0:00 to 23:00
seq_length = 24

# # Create sequences
# x, y = create_sequences(data.values, seq_length)

# # Split the data, 80% for training and 20% for testing
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=True)
# # Split training data into training and validation sets (80% for training and 20% for validation)
# x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, shuffle=True)


# Split the data, 80% for training and 20% for testing
train_data, test_data = train_test_split(data.values, test_size=0.2, shuffle=True)
# Split the training data into training and validation sets (80% for training and 20% for validation)
train_data, val_data = train_test_split(train_data, test_size=0.2, shuffle=True)

# Create sequences from the split data
x_train, y_train = create_sequences(train_data, seq_length)
x_val, y_val = create_sequences(val_data, seq_length)
x_test, y_test = create_sequences(test_data, seq_length)

# Define the model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(seq_length, len(features)), return_sequences=True))
model.add(LSTM(50, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(2)))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

# Train the model
model.fit(x_train, y_train, epochs=800, batch_size=32, validation_data=(x_val, y_val), callbacks=[early_stopping], verbose=visualize_training)

# Save the model
model.save('trained_model/station_1_model.h5', include_optimizer=False)

# Predict using the model
model = load_model(f'trained_model/station_1_model.h5')
predictions = model.predict(x_test)

# Inverse transform the predictions and actual values
inverse_predictions = []
for prediction in predictions:
    inverse_predictions.append(target_scaler.inverse_transform(prediction))
predictions = np.array(inverse_predictions).reshape(-1,2)
inverse_y_test = []
for y in y_test:
    inverse_y_test.append(target_scaler.inverse_transform(y))
y_test = np.array(inverse_y_test).reshape(-1,2)

# Calculate MAE and R2 score for both bikes and docks
mae_bikes = mean_absolute_error(y_test[:, 0], predictions[:, 0])
r2_score_bikes = r2_score(y_test[:, 0], predictions[:, 0])
mae_docks = mean_absolute_error(y_test[:, 1], predictions[:, 1])
r2_score_docks = r2_score(y_test[:, 1], predictions[:, 1])

visualize_prediction(y_test, predictions, mae_bikes, r2_score_bikes, mae_docks, r2_score_docks)


In [8]:
# Load dataset
data = pd.read_csv(f'training_data/station_1.csv', parse_dates=['time'], index_col='time')

# Features and target variables
features = ['hour_sin', 'hour_cos','temperature_celsius', 'relative_humidity_percent', 'barometric_pressure_hpa', 'is_holiday', 'is_weekend']
targets = ['available_bikes', 'available_docks']

# Normalize the features
scaler = MinMaxScaler()
data[features] = scaler.fit_transform(data[features])

# Normalize the targets
target_scaler = MinMaxScaler()
data[targets] = target_scaler.fit_transform(data[targets])

# Move target columns to the last
data = data[[col for col in data.columns if col not in targets] + targets]

In [9]:
data

,temperature_celsius,relative_humidity_percent,barometric_pressure_hpa,is_holiday,is_weekend,hour_sin,hour_cos,available_bikes,available_docks
time,,,,,,,,,
2024-12-01 00:00:00,0.961733,0.654483,0.449139,0.0,1.0,0.500000,1.000000,0.645161,0.322581
2024-12-01 01:00:00,0.961733,0.654483,0.449139,0.0,1.0,0.635214,0.981372,0.645161,0.322581
2024-12-01 02:00:00,0.961733,0.654483,0.449139,0.0,1.0,0.760399,0.926869,0.645161,0.322581
2024-12-01 03:00:00,0.867755,0.738454,0.412359,0.0,1.0,0.866272,0.840534,0.645161,0.354839
2024-12-01 04:00:00,0.867755,0.738454,0.412359,0.0,1.0,0.944980,0.728769,0.645161,0.354839
...,...,...,...,...,...,...,...,...,...
2024-12-31 19:00:00,0.707935,0.722648,0.387674,0.0,0.0,0.055020,0.728769,0.129032,0.870968
2024-12-31 20:00:00,0.675858,0.751050,0.383035,0.0,0.0,0.133728,0.840534,0.129032,0.870968
2024-12-31 21:00:00,0.655599,0.768091,0.379390,0.0,0.0,0.239601,0.926869,0.096774,0.903226


In [10]:
744/24

31.0